In [1]:
!nvidia-smi

Mon Aug  8 18:39:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

Mounted at /content/gdrive
     |████████████████████████████████| 4.7 MB 34.1 MB/s 
     |████████████████████████████████| 6.6 MB 59.1 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 316 kB 37.0 MB/s 
     |████████████████████████████████| 1.4 MB 62.3 MB/s 
     |████████████████████████████████| 233 kB 65.3 MB/s 
     |████████████████████████████████| 175 kB 36.1 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 235 kB 54.6 MB/s 


In [3]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono fonts-texgyre
  javascript-common libcupsfilters1 libcupsimage2 libgs9 libgs9-common
  libijs-0.35 libjbig2dec0 libjs-jquery libkpathsea6 libpotrace0 libptexenc1
  libruby2.5 libsynctex1 libtexlua52 libtexluajit2 libzzip-0-13 lmodern
  poppler-data preview-latex-style rake ruby ruby-did-you-mean ruby-minitest
  ruby-net-telnet ruby-power-assert ruby-test-unit ruby2.5
  rubygems-integration t1utils tex-common tex-gyre texlive-base
  texlive-binaries texlive-latex-base texlive-latex-extra
  texlive-latex-recommended texlive-pictures tipa
Suggested packages:
  fonts-noto apache2 | lighttpd | httpd poppler-utils ghostscript
  fonts-japanese-mi

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

In [5]:
from transformers import AutoConfig, AutoTokenizer, TFAutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import TFBertModel, TFBertForSequenceClassification, BertConfig, TFAutoModelForSequenceClassification
from transformers import glue_convert_examples_to_features
from transformers import InputExample, InputFeatures
import tensorflow as tf


In [6]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")



In [7]:
class InputExample:
    """ A single example for simple sequence classification. """

    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


def make_examples(tokenizer, x, y=None, maxlen=128, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)

    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = int(_y)
        _x = str(_x)
        if isinstance(_x, str):
            text_a = _x
            text_b = ""
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]
        
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    
    features = glue_convert_examples_to_features(
        examples, 
        tokenizer, 
        maxlen, 
        output_mode=output_mode, 
        label_list=list(np.unique(y)))

    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []

    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)

    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features
    
    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels

    return [xdata, ydata], features

In [8]:

gdrive_path = '/content/gdrive/My Drive/ColabNotebooks/'

Test = 'test_set_text.csv'
Train = 'train_set_text.csv'

train_df = pd.read_csv(gdrive_path + Train)
test_df = pd.read_csv(gdrive_path + Test)
print(train_df.columns)
x_train, y_train = train_df['text'].values, train_df['target'].values
x_test, y_test = test_df['text'].values, test_df['target'].values
print(type(x_train))
# print(type(x_train[0]))

Index(['Unnamed: 0', 'text', 'label', 'target'], dtype='object')
<class 'numpy.ndarray'>


In [9]:
train_dataset_base, train_examples = make_examples(tokenizer, x_train, y_train, maxlen=256)
valid_dataset_base, valid_examples = make_examples(tokenizer, x_test, y_test, maxlen=256)


  0%|          | 0/8799 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


  0%|          | 0/8799 [00:00<?, ?it/s]

  0%|          | 0/2172 [00:00<?, ?it/s]

  0%|          | 0/2172 [00:00<?, ?it/s]

In [10]:
for value in train_dataset_base.take(1):
    print(f'     input_ids: {value[0]["input_ids"]}')
    print(f'attention_mask: {value[0]["attention_mask"]}')
    print(f'token_type_ids: {value[0]["token_type_ids"]}')
    print(f'        target: {value[1]}')
    break

     input_ids: [    2  9790  3788 48208 25024  2786  3237  2791  3753  8255  1379  5433
  3457  2871  1348  3398  2791  8324  3381  2831  5824  2799  4247  1379
  9790  2802  3788  2789 55950 13997  6994  8050  3168  1012  2786  2802
  3016  5198  2791  3753  5279  2921  3332  3445  3811  1379  2831  5824
  2799  9790  2802  3788  4860  6994  2803  3096  5704  1012  2789  3361
  7245  2789  4134  2791  4531 30358  5347  4237 10441  1348  3099  5698
  5367  4080 25024  1379  8529  5241  2791  3525  8529 18815  2803  2789
  2802  2930  6140  2830  1348  3111  7238  8529  9009  2786  5698  8617
  1379  6140  3402  2817  1012 10874  2806  1348  3960  3371 10434  2791
  9790 48208  8529  2786  5698  4482  4338  2806  1012  3753  5279  1348
 13004  1348 11441  1348  5073  1348  7234  1348  7902  4548  5279  1348
  9541  1348  9723  2921  2847  2803  2831  5824  2799  2802  3788  4788
  3991  1012  7902  4548  5279  1348  9541  1379  9723  5367  4080 25024
  1379  8529  2803  2791  4610  134

In [11]:
def build_model(model_name, config, learning_rate=3e-5):
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    return model

In [12]:
train_dataset_base = train_dataset_base.batch(10)
valid_dataset_base = valid_dataset_base.batch(10)

MODEL_NAME = 'HooshvareLab/bert-fa-base-uncased'

# model = TFBertForSequenceClassification.from_pretrained(model_name, config=config)

labels = list(train_df['label'].unique())
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')


config = AutoConfig.from_pretrained(
    MODEL_NAME, **{
        'label2id': label2id,
        'id2label': id2label,
    })
model = build_model(MODEL_NAME, config, learning_rate=1e-4)
# r = model.fit(
    # train_dataset_base,
    # validation_data=valid_dataset_base,
    # epochs=1,
    # verbose=1)

# final_accuracy = r.history['val_accuracy']
# print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))


label2id: {'اقتصادي': 0, 'فرهنگي': 1, 'علمي': 2, 'سياسي': 3, 'ورزشي': 4, 'آموزشي': 5, 'فقه و حقوق': 6, 'بهداشتي': 7, 'تاريخي': 8, 'مذهبي': 9, 'اجتماعي': 10}
id2label: {0: 'اقتصادي', 1: 'فرهنگي', 2: 'علمي', 3: 'سياسي', 4: 'ورزشي', 5: 'آموزشي', 6: 'فقه و حقوق', 7: 'بهداشتي', 8: 'تاريخي', 9: 'مذهبي', 10: 'اجتماعي'}


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
r = model.fit(
    train_dataset_base,
    validation_data=valid_dataset_base,
    epochs=3,
    verbose=1)

final_accuracy = r.history['val_accuracy']
print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))


Epoch 1/3
880/880 [==============================] - 580s 637ms/step - loss: 0.8094 - accuracy: 0.7684 - val_loss: 0.5189 - val_accuracy: 0.8656
Epoch 2/3
880/880 [==============================] - 559s 635ms/step - loss: 0.4775 - accuracy: 0.8729 - val_loss: 0.6192 - val_accuracy: 0.8131
Epoch 3/3
880/880 [==============================] - 559s 635ms/step - loss: 0.3508 - accuracy: 0.9076 - val_loss: 0.5995 - val_accuracy: 0.8554
FINAL ACCURACY MEAN:  0.8446900049845377


In [15]:
!jupyter nbconvert --to pdf /content/gdrive/MyDrive/ColabNotebooks/12.ipynb

[NbConvertApp] Converting notebook /content/gdrive/MyDrive/ColabNotebooks/12.ipynb to pdf
[NbConvertApp] Writing 71336 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 57006 bytes to /content/gdrive/MyDrive/ColabNotebooks/12.pdf
